In [ ]:
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt

import numpy as np
import os

import IPython

In [ ]:
# Load bjj.npz file
path = os.path.join(os.getcwd(), 'bjj.npz')
with np.load(path) as data:
    train_examples = data['train_examples']
    train_labels = data['train_labels']
    test_examples = data['test_examples']
    test_labels = data['test_labels']

In [ ]:
# Concatenate train and test examples to find max for normalization
examples_maxes = np.max(np.concatenate((train_examples, test_examples)), axis=0)

# Normalize train and test examples to values in [0, 1]
train_examples = train_examples.astype('float32') / examples_maxes
test_examples = test_examples.astype('float32') / examples_maxes

In [ ]:
# Save example maxes
np.savez('examples_maxes.npz', examples_maxes=examples_maxes)

In [ ]:
# Model builder for Keras Tuner
def model_builder(hp):
    # Sequential with 12 input nodes
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape = (12,)))
    
    # Tune number of units in first Dense layer
    hp_units_1 = hp.Int('units_1', min_value = 32, max_value = 512, step = 32)
    hp_reg_1 = hp.Choice('reg_1', values = [1e-0, 1e-1, 1e-2, 1e-3, 1e-4])
    model.add(keras.layers.Dense(units = hp_units_1, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_reg_1)))
    
    # Tune dropout rate in Dropout layer
    hp_dropout_rate = hp.Choice('dropout_rate', values = [0.00, 0.25, 0.50, 0.75])
    model.add(keras.layers.Dropout(hp_dropout_rate))
    
    # Tune number of units in second Dense layer
    hp_units_2 = hp.Int('units_2', min_value = 32, max_value = 512, step = 32)
    hp_reg_2 = hp.Choice('reg_2', values = [1e-0, 1e-1, 1e-2, 1e-3, 1e-4])
    model.add(keras.layers.Dense(units = hp_units_2, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_reg_2)))
    
    # Output layer
    model.add(keras.layers.Dense(5))
    
    # Tune learning rate
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                  metrics = ['accuracy'])
    
    return model

In [ ]:
# Instantiate Keras Tuner
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = '.',
                     project_name = 'bjj_kt',
                     overwrite = True)

In [ ]:
# Clear the training output at the end of every training
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [ ]:
# Search for the best hyperparameters
tuner.search(train_examples, train_labels, epochs = 10, validation_data = (test_examples, test_labels), callbacks = [ClearTrainingOutput()])

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

# Output hyperparameter search results
print(f"""
The hyperparameter search is complete. 
The optimal number of units in the first densely-connected layer is {best_hps.get('units_1')}. 
The optimal l2 regulariation in the first densely-connected layer is {best_hps.get('reg_1')}. 
The optimal dropout rate in the dropout layer is {best_hps.get('dropout_rate')}. 
The optimal number of units in the second densely-connected layer is {best_hps.get('units_2')}. 
The optimal l2 regulariation in the second densely-connected layer is {best_hps.get('reg_2')}. 
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

In [ ]:
# Build the model using the found hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model
model.fit(train_examples, train_labels, epochs = 10, validation_data = (test_examples, test_labels), verbose = 2)

In [ ]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_examples, test_labels, verbose = 2)

In [ ]:
# Add a Softmax layer to turn the model into a probability model
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [ ]:
# Predict for all test examples
predictions = probability_model.predict(test_examples)

In [ ]:
# Predict on one example only
predictions_single = probability_model.predict(np.expand_dims(test_examples[1], 0))

print(predictions_single)
print(np.argmax(predictions_single[0]))
print(test_labels[1])

In [ ]:
# Save the model in h5 format
model.save('bjj_model.h5')

In [ ]:
# Test reloading the model and outputing the summary
new_model = tf.keras.models.load_model('bjj_model.h5')
new_model.summary()

In [ ]:
# Test evaluate on the loaded model
loss, acc = new_model.evaluate(test_examples, test_labels, verbose = 2)

In [ ]:
# Convert the loaded model to a probability model
new_probability_model = tf.keras.Sequential([new_model, tf.keras.layers.Softmax()])

In [ ]:
# Test probability model on one test example only
predictions_single = new_probability_model.predict(np.expand_dims(test_examples[1], 0))

print(predictions_single)
print(np.argmax(predictions_single[0]))
print(test_labels[1])